### Extract data from csv

In [1]:
# Import libraries for data manipulation
import pandas as pd
import numpy as np
import mysql.connector
import os

In [2]:
# Load raw data from csv file
df = pd.read_csv('data_sample.csv', encoding='iso-8859-1')

In [3]:
# Take a look of the dataframe
df.head()

,ï»¿Date,Categories,Description,Amount,Person
0,01/29/2023,Transport,Uber,80.00,Nicole
1,01/29/2023,Dineout,Dinner,44.00,Kevin
2,01/29/2023,Groceries,Metro,82.85,Nicole
3,01/30/2023,Groceries,Metro,121.64,Nicole
4,01/30/2023,Groceries,Dollarama,11.30,Nicole


### Transform data

In [4]:
# Rename columns to match the database schema
df.rename(columns={'ï»¿Date':'TransactionDate', 'Categories':'CategoryID', 'Person':'UserID'}, inplace=True)

In [5]:
# Convert Categories into CategoryID
mapping = {'Income': 1, 'Groceries ': 2, 'Transport': 3, 'Dineout': 4, 'Beauty': 5, 'Furniture': 6, 'Household': 7, 'Kitchen': 8,
           'Rent': 9, 'Entertainment': 10, 'Miscellaneous': 11}
df['CategoryID'] = df['CategoryID'].map(mapping)

In [6]:
# Convert User into UserID
mapping = {'Kevin': 1, 'Nicole': 2}
df['UserID'] = df['UserID'].map(mapping)

In [7]:
# Convert TransactionDate into datetime
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format='%m/%d/%Y')

In [8]:
# Review the transformed data
df.head()

,TransactionDate,CategoryID,Description,Amount,UserID
0,2023-01-29,3,Uber,80.00,2
1,2023-01-29,4,Dinner,44.00,1
2,2023-01-29,2,Metro,82.85,2
3,2023-01-30,2,Metro,121.64,2
4,2023-01-30,2,Dollarama,11.30,2


### Load data into MySQL database

In [9]:
# Get keys from environment variables
myhost = os.environ.get('MYSQLHOST')
myuser = os.environ.get('MYSQLUSER')
mypassword = os.environ.get('MYSQLPASS')
my_db = 'householdfinance'

# Establish connection to the database
mydb = mysql.connector.connect(
    host=myhost,
    user=myuser,
    password=mypassword,
    database=my_db
)

In [10]:
# Create a cursor
cursor = mydb.cursor()

In [11]:
# define the query
sql_query = 'INSERT INTO transactions (TransactionDate, CategoryID, Description, Amount, UserID) VALUES (%s, %s, %s, %s, %s)'

In [12]:
# define the values to be inserted
values = list(df.itertuples(index=False, name=None))

for value in values:
    cursor.execute(sql_query, value)

# Commit the changes
mydb.commit()

In [13]:
# Close the cursor and the connection
cursor.close()
mydb.close()